In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# The forward and backward passes

In [2]:
#export
from exp.nb_01 import *

In [3]:
#export
def x_to_tensor(listed):
    return tensor(list(map(array, listed)), dtype=torch.float32).reshape(len(listed), -1)

def y_to_tensor(listed):
    return tensor(array(listed))

In [4]:
#export
def get_data():
    mnist = DataBlock(blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
                     get_items = get_image_files,
                     splitter = GrandparentSplitter('training', 'testing'),
                     get_y = parent_label)
    dls = mnist.dataloaders(untar_data(URLs.MNIST))
    
    x_train, y_train, x_valid, y_valid = [], [], [], []
    
    for elem in dls.train_ds:
        x_train.append(elem[0])
        y_train.append(elem[1])
    for elem in dls.valid_ds:
        x_valid.append(elem[0])
        y_valid.append(elem[1])
        
    return x_to_tensor(x_train), y_to_tensor(y_train), x_to_tensor(x_valid), y_to_tensor(y_valid)
    
def normalize(x, m, s): return (x-m)/s

In [5]:
%time x_train, y_train, x_valid, y_valid = get_data()

CPU times: user 48.7 s, sys: 2.85 s, total: 51.6 s
Wall time: 1min 7s


In [6]:
x_train, x_valid = x_train / 255.0, x_valid / 255.0

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1307), tensor(0.3081))

In [8]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [9]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-5.9790e-09), tensor(1.0000))

In [10]:
#export
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f'Not Near zero: {a}'

In [11]:
test_near_zero(train_mean)
test_near_zero(1 - train_std)

In [12]:
n, m = x_train.shape
c = y_train.max() + 1
n, m, c

(60000, 784, tensor(10))

# Foundations version

## Basic architecture

In [13]:
# num hidden
nh = 50

In [14]:
# standard xavier init
w1 = torch.randn(m,nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [15]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1 / math.sqrt(m))

In [16]:
w1.std(), 1 / math.sqrt(m)

(tensor(0.0357), 0.03571428571428571)

In [17]:
x_valid.mean(), x_valid.std() # ~ 0, 1

(tensor(0.0060), tensor(1.0077))

In [18]:
def lin(x, w, b): return x@w + b

In [19]:
t = lin(x_valid, w1, b1)

In [20]:
t.mean(), t.std() # ~ 0, 1

(tensor(0.0306), tensor(0.9923))

In [21]:
def relu(x): return x.clamp_min(0.)

In [22]:
t = relu(lin(x_valid, w1, b1))

In [23]:
t.mean(), t.std() # ~ pretty bad thanks to the relu

(tensor(0.4064), tensor(0.5969))

In [24]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh) * math.sqrt(2/m)

In [25]:
w1.mean(), w1.std()

(tensor(3.5494e-05), tensor(0.0503))

In [26]:
t = relu(lin(x_valid, w1, b1))

In [27]:
t.mean(), t.std()

(tensor(0.6344), tensor(0.8869))

In [28]:
# export
from torch.nn import init

In [29]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # fan_out because in pytorch the tensors are batchsize first!
t = relu(lin(x_valid, w1, b1))

In [30]:
w1.mean(), w1.std()

(tensor(-2.0494e-05), tensor(0.0504))

In [31]:
t.mean(), t.std()

(tensor(0.4829), tensor(0.7551))

In [32]:
w1.shape

torch.Size([784, 50])

In [33]:
init.kaiming_normal_??

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \frac{\text{gain}}{\sqrt{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``

In [34]:
import torch.nn

In [35]:
m, nh

(784, 50)

In [36]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [37]:
# what if...
def relu(x): return x.clamp_min(0.) - 0.5

In [38]:
w1 = torch.randn(m,nh) * math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0334), tensor(0.8095))

In [39]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [40]:
%timeit -n 10 _ = model(x_valid)

9.43 ms ± 661 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

# Loss function: MSE

In [42]:
model(x_valid).shape

torch.Size([10000, 1])

In [43]:
#export
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [44]:
y_train, y_valid = y_train.float(), y_valid.float()

In [45]:
preds = model(x_train)

In [46]:
preds.shape

torch.Size([60000, 1])

In [47]:
mse(preds, y_train)

tensor(27.4242)

# Gradients and backward pass

In [48]:
def mse_grad(inp, targ):
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [49]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp > 0).float() * out.g

In [50]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.T
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [51]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we dont actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [52]:
forward_and_backward(x_train, y_train)

In [53]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [54]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [55]:
def forward(inp, targ):
    # forward pass
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we dont actually need the loss in backward!
    return mse(out, targ)

In [56]:
loss = forward(xt2, y_train)

In [57]:
loss.backward()

In [58]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

## Refactor model

In [59]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5 # better(?) version of relu
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [60]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.T
        # Creating a giant outer product just to sum it is inefficient
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [61]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2 * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [62]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [63]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1, b1, w2, b2)

In [64]:
%time loss = model(x_train, y_train)

CPU times: user 255 ms, sys: 0 ns, total: 255 ms
Wall time: 66.8 ms


In [65]:
%time model.backward()

CPU times: user 10.5 s, sys: 2.61 s, total: 13.1 s
Wall time: 3.64 s


In [66]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## Module.foward()

In [89]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [90]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

In [91]:
class Lin(Module):
    def __init__(self, w,  b): 
        self.w, self.b = w, b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp): 
        inp.g = out.g @ self.w.T
        self.w.g = torch.einsum('bi,bj->ij', inp, out.g)
        self.b.g = out.g.sum(0)

In [92]:
class Mse(Module):
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [93]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [94]:
w1.g, b1.g, w2.g, b2.g = [None] * 4

In [95]:
%time model = Model()

CPU times: user 10 µs, sys: 1e+03 ns, total: 11 µs
Wall time: 15.3 µs


In [96]:
%time loss = model(x_train, y_train)

CPU times: user 199 ms, sys: 0 ns, total: 199 ms
Wall time: 51.1 ms


In [97]:
%time model.backward()

CPU times: user 358 ms, sys: 106 ms, total: 464 ms
Wall time: 129 ms


In [99]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

In [108]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
    
    def forward(self, inp): 
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.T
        self.w.g = inp.T @ out.g
        self.b.g = out.g.sum(0)

In [109]:
w1.g, b1.g, w2.g, b2.g = [None] * 4

In [110]:
%time model = Model()

CPU times: user 11 µs, sys: 1e+03 ns, total: 12 µs
Wall time: 14.8 µs


In [111]:
%time loss = model(x_train, y_train)

CPU times: user 213 ms, sys: 820 µs, total: 213 ms
Wall time: 53 ms


In [112]:
%time model.backward()

CPU times: user 307 ms, sys: 91 ms, total: 398 ms
Wall time: 99.5 ms


In [113]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## nn.Linear and nn.Module

In [115]:
#export
from torch import nn

In [116]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
         
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [117]:
model = Model(m, nh, 1)

In [118]:
%time loss = model(x_train, y_train)

CPU times: user 198 ms, sys: 147 µs, total: 198 ms
Wall time: 57.3 ms


In [120]:
%time loss.backward()

CPU times: user 204 ms, sys: 1.05 ms, total: 205 ms
Wall time: 52.3 ms


# Export

In [1]:
!python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py


# Selfmade Model

In [124]:
x_valid.shape, y_valid.shape

(torch.Size([10000, 784]), torch.Size([10000]))

In [125]:
nh = 50

In [126]:
m = x_valid.shape[1]

In [141]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1) * math.sqrt(2/m)
b2 = torch.zeros(1)

In [142]:
def lin(x, w, b): return x @ w + b

In [143]:
def relu(x): return x.clamp_min(0.) - 0.5

In [144]:
t = relu(lin(x_valid, w1, b1))

In [145]:
t.mean(), t.std()

(tensor(0.1369), tensor(0.8892))

In [146]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    
    return out

In [147]:
%timeit -n 10 _ = model(x_valid)

7 ms ± 772 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Loss function

In [149]:
def mse(output, target): return (output.squeeze() - target).pow(2).mean()

In [150]:
preds = model(x_valid)

In [153]:
preds.shape

torch.Size([10000, 1])

In [154]:
y_valid.shape

torch.Size([10000])

In [152]:
mse(preds, y_valid)

tensor(29.3897)

## Gradient and backward pass

In [166]:
def mse_grad(inp, targ):  
    inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [167]:
def relu_grad(inp, out):
    inp.g = (inp > 0).float() * out.g

In [168]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.T
    w.g = inp.T @ out.g
    b.g = out.g.sum(0)

In [169]:
def forward_and_backward(inp, targ):
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    loss = mse(out, targ)
    
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b2)

In [170]:
forward_and_backward(x_train, y_train)

In [171]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [172]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    
    return mse(out,targ)

In [173]:
loss = forward(xt2, y_train)

In [174]:
loss.backward()

In [175]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [178]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp @ self.w + self.b # maybe make self.inp to inp?
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.T
        self.w.g = self.inp.T @ self.out.g
        self.b.g = self.out.g.sum(0)

In [179]:
y_train.shape

torch.Size([60000])

In [180]:
class Mse():
    def __call__(self, inp, targ):
        self.inp, self.targ = inp, targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2 * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [181]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        self.loss(x, targ)
        
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [182]:
w1.g, b1.g, w2.g, b2.g = [None] * 4

In [183]:
model = Model(w1, b1, w2, b2)

In [184]:
%time loss = model(x_valid, y_valid)

CPU times: user 50.6 ms, sys: 1.43 ms, total: 52 ms
Wall time: 12.9 ms


In [185]:
%time model.backward()

CPU times: user 80.3 ms, sys: 0 ns, total: 80.3 ms
Wall time: 20.2 ms


# Module.forward()

In [206]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
        
    def backward(self): self.bwd(self.out, *self.args)

In [207]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    
    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

In [208]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def forward(self, inp): return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.T
        self.w.g = inp.T @ out.g
        self.b.g = out.g.sum(0)

In [209]:
class Mse(Module):
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    
    def bwd(self, out, inp, targ): inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0] 

In [210]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [214]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [215]:
%time loss = model(x_train, y_train)

CPU times: user 197 ms, sys: 0 ns, total: 197 ms
Wall time: 50.4 ms


In [216]:
%time model.backward()

CPU times: user 283 ms, sys: 124 ms, total: 407 ms
Wall time: 102 ms
